In [1]:
import pandas as pd
import numpy as np
import re

def readVectors_test(filePath):
    vectors = []
    utt = []
    with open(filePath, "r") as f:
        lignes = f.readlines()
        line_idx = 0
        last_printed_percent = -1
        number_of_lines = len(lignes)

        for ligne in lignes:
            if ligne:
                match = re.match(r'^(\S+)\s+\[([\d\s.]+)]$', ligne)
                if match:
                    identifiant, elements_str = match.group(1), match.group(2)
                    elements = np.array([float(e) for e in elements_str.split()])
                    utt.append(identifiant)
                    vectors.append(elements)

                    # Afficher la progression
                    line_idx += 1
                    percent = int((line_idx / number_of_lines) * 100)
                    if percent % 10 == 0 and percent != last_printed_percent:
                        last_printed_percent = percent
                else:
                    print(f"Erreur à la ligne {line_idx} : {ligne}")

    return utt, np.array(vectors)

In [85]:
open_smile = pd.read_csv("./data/vox1_opensmile.csv")
df = pd.read_csv("./data/df_binary.csv")
open_smile.head()

,Unnamed: 0,name,F0semitoneFrom27.5Hz_sma3nz_amean,F0semitoneFrom27.5Hz_sma3nz_stddevNorm,F0semitoneFrom27.5Hz_sma3nz_percentile20.0,F0semitoneFrom27.5Hz_sma3nz_percentile50.0,F0semitoneFrom27.5Hz_sma3nz_percentile80.0,F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2,F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope,...,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,spectralFluxUV_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp
0,0,/local_disk/arges/jduret/corpus/voxceleb1/dev/...,28.092182,0.061502,26.984283,27.878792,28.929113,1.944830,105.241852,112.295776,...,0.034317,-0.012570,0.370522,5.178792,3.473946,0.163571,0.134887,0.103571,0.084440,-21.141186
1,1,/local_disk/arges/jduret/corpus/voxceleb1/dev/...,27.010643,0.060816,25.892637,26.687866,28.496979,2.604341,116.249120,262.780430,...,0.031585,-0.012621,0.323391,4.561403,2.237927,0.319474,0.322906,0.120000,0.082248,-19.915205
2,2,/local_disk/arges/jduret/corpus/voxceleb1/dev/...,26.186111,0.070707,24.654234,26.461887,27.661386,3.007153,138.503590,150.486820,...,0.021272,-0.009637,0.365133,5.187836,3.429603,0.166316,0.108977,0.104211,0.068080,-21.199615
3,3,/local_disk/arges/jduret/corpus/voxceleb1/dev/...,27.494238,0.095222,25.322489,27.320807,29.987375,4.664886,152.163700,185.322500,...,0.010894,0.002566,0.137623,5.098494,2.917153,0.187200,0.116534,0.141667,0.136646,-27.372116
4,4,/local_disk/arges/jduret/corpus/voxceleb1/dev/...,27.532742,0.136942,24.791954,26.543228,30.337467,5.545513,32.876427,22.981077,...,0.013177,-0.002150,0.146955,4.537815,3.565365,0.167619,0.128319,0.098000,0.052593,-27.439383


In [ ]:
import csv
id_vox, df_binary = readVectors_test("./vox1-transformed.txt")
count = 0
def is_match(id, row):
    if id == row["name"].replace("", "").replace(
                                "/local_disk/arges/jduret/corpus/voxceleb1/test/wav/", "").replace(
                                "/local_disk/arges/jduret/corpus/voxceleb1/dev/wav/", "").replace(".wav", "").replace(
                                "/", "-"):
        return True
    else:
        return False
for i in range(2,df_binary.shape[1]):
    with open(f"./data/BA/BA{i}_0.csv", mode='w+', newline='') as ba0:
        with open(f"./data/BA/BA{i}_1.csv", mode='w+', newline='') as ba1:
            columns = ['name'] + open_smile.columns[2:].tolist()
            writer_ba0 = csv.writer(ba0)
            writer_ba1 = csv.writer(ba1)
            writer_ba0.writerow(columns)
            writer_ba1.writerow(columns)
            for id, vec in zip(id_vox, df_binary):
                if vec[i] == 0:
                    mask = open_smile["name"].str.replace("/local_disk/arges/jduret/corpus/voxceleb1/test/wav/", "").replace("/local_disk/arges/jduret/corpus/voxceleb1/dev/wav/", "").replace(".wav", "").replace("/", "-").str.contains(id)
                    smile_vec = open_smile.loc[mask].iloc[:, 2:]
                    for _, row in smile_vec.iterrows():
                        writer_ba0.writerow([id] + row.tolist())
                elif vec[i] == 1:
                    mask = open_smile["name"].str.replace("/local_disk/arges/jduret/corpus/voxceleb1/test/wav/", "").replace("/local_disk/arges/jduret/corpus/voxceleb1/dev/wav/", "").replace(".wav", "").replace("/", "-").str.contains(id)
                    smile_vec = open_smile.loc[mask].iloc[:, 2:]
                    for _, row in smile_vec.iterrows():
                        writer_ba1.writerow([id] + row.tolist())

In [1]:
BA = ['BA' + str(i) for i in range(df_binary.shape[1])]

NameError: name 'df_binary' is not defined

In [84]:
print("id10271-1gtz-CUIygI-00011".replace("-", "/"))

# print(.replace("-", "/").split("/"))

text = "/local_disk/arges/jduret/corpus/voxceleb1/test/wav/id10271/1gtz-CUIygI/00011.wav".replace("", "").replace("/local_disk/arges/jduret/corpus/voxceleb1/test/wav/", "").replace("/local_disk/arges/jduret/corpus/voxceleb1/dev/wav/", "").replace(".wav", "")
text = text.replace("/", "-")

print(text)

In [83]:
features_vox1_test=pd.read_csv("./data/vox1_opensmile.csv")
features_vox1_test.drop('Unnamed: 0', axis=1, inplace=True)
df.drop('Unnamed: 0', axis=1, inplace=True)
df = df.rename(columns={0: 'name'})

# Créer les fichiers BA
for i in range(1, df.shape[1]):
    print(f"BA{i-1}")
    print(df[f"BA{i-1}"])
    ba0 = df[df[f"BA{i-1}"]==0]["name"]
    ba1 = df[df[f"BA{i-1}"]==1]["name"]
    
    ba0 = pd.merge(ba0, features_vox1_test, on='name', how='inner')
    ba1 = pd.merge(ba1, features_vox1_test, on='name', how='inner')
    #ba0 = pd.DataFrame([df_binary["name"], np.where(df_binary[f"BA{i-1}"]==0, 1, 0)], axis = 1) # colonne i 
    #ba1 = pd.DataFrame([df_binary["name"], df_binary[f"BA{i-1}"]], axis = 1)
    ba0.to_csv(f"./data/BA/BA{i-1}_0.csv", index = False)
    ba1.to_csv(f"./data/BA/BA{i-1}_1.csv", index = False)